### Challenge 1

In [3]:
# challenge 1

# instructions are unclear...

import base64
from base64 import b64encode 
import binascii

# Convert hex to base64
# indicates this string to be converted to base64
myHex = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
targetBase64 = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'


# This is not "a string"; it is a hex-encoded string.  I thought it was a string representing hex data.

# So insted we decode the string and convert that string to base64

# parse hex string into sequence of bytes
decoded_str_bytes = bytes.fromhex(myHex)
print(f'{decoded_str_bytes=}')
print(f'{decoded_str_bytes.decode("utf-8")=}')

b64 = b64encode(decoded_str_bytes)
print(f'{b64=}')

base64string = b64.decode('utf-8')
print(f'{base64string=}')

assert targetBase64 == base64string, base64string


decoded_str_bytes=b"I'm killing your brain like a poisonous mushroom"
decoded_str_bytes.decode("utf-8")="I'm killing your brain like a poisonous mushroom"
b64=b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'
base64string='SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'


### Challenge 2

In [30]:
# challenge 2
# this time what the challenge calls a string is actually a hex value.

# string to int type conversions
s1 = int('1c0111001f010100061a024b53535009181c', 16)
s2 = int('686974207468652062756c6c277320657965', 16)

target = '746865206b696420646f6e277420706c6179'

# do the xor, convert int to string
candidate = f'{s1^s2:x}'

assert target == candidate, candidate

# treat hex values as ascii values and map to characters
bytes.fromhex(target).decode('utf-8')


"the kid don't play"

### Challenge 3

In [108]:
# from the start, the instructions are much more precise
encrypted = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
encrypted_bytes = bytes.fromhex(encrypted)
num_bytes = len(encrypted_bytes)
print(f'{num_bytes=}')
encrypted_hex = int(encrypted, 16)
print(encrypted_bytes)

num_bytes=34
b'\x1b77316?x\x15\x1b\x7f+x413=x9x(7-6<x7>x:9;76'


In [103]:
from collections import Counter
import string

vocab = string.ascii_lowercase + ' '# + string.ascii_uppercase
def english_score(txt):
    counter = Counter(txt)
    
    # txt gets a point for common alphabetic characters
    most = counter.most_common()
    score = sum(1 if tup[0] in vocab else -1 for tup in most)
    return score

def english_score2(txt):
    counter = Counter(txt)
    
    # txt gets a point for common alphabetic characters
    most = counter.most_common()
    score = sum(val*tup[1] if tup[0] in vocab else -val*tup[1] 
                for tup, val in zip(most,range(len(most),1,-1)))
    return score

english_score2('aaa;')

6

In [133]:


def decrypt_using_single_byte(encrypted_txt):
    encrypted_bytes = bytes.fromhex(encrypted)
    num_bytes = len(encrypted_bytes)
    #print(f'{num_bytes=}')
    encrypted_hex = int(encrypted, 16)
    #print(encrypted_bytes)

    scores = []
    scores2 = []
    
    for single_byte in range(0,256):
        #print()
        #print(f'{single_byte=:x}')


        ######
        # This doesn't work because UTF-8 encoding becomes
        # two bytes for codes above 127
        # turn string to bytes, to hex string, to int
        ######
        #key = chr(single_byte)*num_bytes
        #print(f'{key=}')
        #keyHex = int(key.encode('utf-8').hex(), 16)

        ######
        # Instead, construct the bytes directly
        ######
        key = [single_byte] * num_bytes
        keyHex = int(bytes(key).hex(), 16)


        #print(f'{keyHex=}')
        #print(f'{len(str(keyHex))=}')

        decrypted_hex = encrypted_hex ^ keyHex
        #print(decrypted_hex)

        # ensure the length of the hex string matches the length of the
        # encrypted bytes, otherwise leading 0s get dropped and cause exceptions
        decrypted_hex_str = f'{decrypted_hex:0{num_bytes*2}x}'
        #print(f'{decrypted_hex_str=}')
        #print(f'{len(decrypted_hex_str)=}')

        ######
        # For the same reason above, converting to a hex string and
        # decoding as utf-8 doesn't work
        ######
        #ptxt = bytes.fromhex(decrypted_hex_str).decode('utf-8')


        ptxt_bytes = bytes.fromhex(decrypted_hex_str)
        ptxt = ''
        for b in ptxt_bytes:
            ptxt += chr(b)
        #print(f'{ptxt=}')


        scores.append((ptxt, english_score(ptxt)))
        scores2.append((ptxt, english_score2(ptxt)))
        
    def sorter(i):
        return i[1]
    scores.sort(key=sorter, reverse=True)
    scores2.sort(key=sorter, reverse=True)

    return scores[0], scores, scores2

In [134]:
encrypted = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
_, scores, scores2 = decrypt_using_single_byte(encrypted)

for tup in scores[:10]:
    print(tup)
print()
for tup in scores2[:10]:
    print(tup)
    

("Cooking MC's like a pound of bacon", 13)
("Dhhlni`'JD t'knlb'f'whric'ha'efdhi", 11)
('Ieeacdm*GI-y*fcao*k*ze\x7fdn*el*hkied', 9)
('Bnnjhof!LB&r!mhjd!`!qntoe!ng!c`bno', 9)
('Ammikle"OA%q"nkig"c"rmwlf"md"`caml', 9)
('@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm', 9)
('Gkkomjc$IG#w$hmoa$e$tkqj`$kb$fegkj', 9)
('Fjjnlkb%HF"v%iln`%d%ujpka%jc%gdfjk', 9)
('Eiimoha&KE!u&jomc&g&vishb&i`&dgeih', 9)
('Uyy}\x7fxq6[U1e6z\x7f}s6w6fycxr6yp6twuyx', 7)

("Cooking MC's like a pound of bacon", 342)
('@llhjmd#N@$p#ojhf#b#slvmg#le#ab`lm', 114)
('Ammikle"OA%q"nkig"c"rmwlf"md"`caml', 110)
("Dhhlni`'JD t'knlb'f'whric'ha'efdhi", 110)
('Eiimoha&KE!u&jomc&g&vishb&i`&dgeih', 108)
('Gkkomjc$IG#w$hmoa$e$tkqj`$kb$fegkj', 106)
('Ieeacdm*GI-y*fcao*k*ze\x7fdn*el*hkied', 104)
('Fjjnlkb%HF"v%iln`%d%ujpka%jc%gdfjk', 100)
('Kggcafo(EK/{(dacm(i(xg}fl(gn(jikgf', 86)
('Nbbfdcj-@N*~-adfh-l-}bxci-bk-olnbc', 84)


### Challenge 4

In [135]:
top_res = (None,0)
with open('4.txt') as f:
    for encrypted in f:
        res, _, _ = decrypt_using_single_byte(encrypted)
        if res[1] > top_res[1]:
            top_res = res

print(top_res)

('Now that the party is jumping\n', 15)


### Challenge 5

In [194]:
### Questionable instructions again.  The space at the end of the first line is NOT to be encrypted, but the carriage return is.  However,
### the carriage return in the expected encryption should be ignored.  That is, there is no one-to-mapping between the plaintext lines
### and the encrypted lines.  Would have been clearer if the expected encryption was presented as a single line of text.

ptxt = """Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal"""
key = "ICE"
expected = \
    "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63" \
    "343c2a26226324272765272a282b2f20430a652e2c652a3124333" \
    "a653e2b2027630c692b20283165286326302e27282f"

print(ptxt)

ptxt_bytes = [ord(c) for c in ptxt ]
key_bytes = [ord(c) for c in key]

ctxt_bytes = []
key_i = 0
for pbyte in ptxt_bytes:
    ctxt_bytes.append(pbyte ^ key_bytes[key_i])
    key_i = (key_i + 1) % len(key_bytes)

out = ''
for cb in ctxt_bytes:
    out += f'{cb:02x}'
print(out)
    
assert expected == out, out

Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal
0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f


### Challenge 6

In [206]:
def hamming_dist(s1, s2):
    assert len(s1) == len(s2), f'{s1}: {len(s1)=}, {s2}: {len(s2)=}'
    s1_bytes = [ord(c) for c in s1]
    s2_bytes = [ord(c) for c in s2]
    #print(s1_bytes)
    
    s1_bits = ''.join([f'{int_val:08b}' for int_val in s1_bytes])
    s2_bits = ''.join([f'{int_val:08b}' for int_val in s2_bytes])
    #print(s1_bits)

    return sum(c1 != c2 for c1,c2 in zip(s1_bits, s2_bits))
hamming_dist('this is a test','wokka wokka!!!')

37